In [1]:
import os
import glob

from keras.applications.resnet50 import ResNet50

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import optimizers

import numpy as np
import pandas as pd

/Users/ys/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/matplotlib/__init__.py:1085: UserWarning: Duplicate key in file "/Users/ys/.matplotlib/matplotlibrc", line #628
  (fname, cnt))


In [2]:
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', 
           '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']

In [3]:
batch_size = 32
nb_classes = len(classes)

img_rows, img_cols = 150, 150
channels = 3

train_data_dir = 'input/processed/train1'
valid_data_dir = 'input/processed/valid'
test_data_dir = 'input/processed/test'

nb_train_samples = 12399 - (15 * 24)
nb_val_samples = 15 * 24
nb_epoch = 10

result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

In [4]:
# モデルと学習済み重みをロード
# Fully-connected層（FC）はいらないのでinclude_top=False）
input_tensor = Input(shape=(img_rows, img_cols, 3))

In [5]:
# create the base pre-trained model
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer
predictions = Dense(nb_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional layers
for layer in base_model.layers:
    layer.trainable = False

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

Found 11799 images belonging to 24 classes.


In [8]:
valid_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

validation_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

Found 360 images belonging to 24 classes.


In [9]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

0 input_1
1 zero_padding2d_1
2 conv1
3 bn_conv1
4 activation_1
5 max_pooling2d_1
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 add_1
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 add_2
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 add_3
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 add_4
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c
58 add_5
59 activation_16
60 res3c_branch2a
61 bn3c_branch2a
62 ac

In [10]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    nb_epoch=nb_epoch,
    validation_data=validation_generator,
    nb_val_samples=nb_val_samples)

/Users/ys/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., epochs=10, validation_steps=360, steps_per_epoch=376)`
  del sys.path[0]


Epoch 1/10
  2/376 [..............................] - ETA: 7153s - loss: nan - acc: 0.0469

KeyboardInterrupt: 

In [ ]:
model.save_weights(os.path.join(result_dir, 'finetuning3.h5'))

In [ ]:
# load test data and make prediction
path = os.path.join('input', 'processed', 'test', '*.jpg')
files = sorted(glob.glob(path))

In [ ]:
test_id = []
test = []

for fl in files:
    flbase = os.path.basename(fl)
    img = image.load_img(fl, target_size=(img_rows, img_cols))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0

    test_id.append(flbase)
    test.append(model.predict(x)[0])    

In [ ]:
df1 = pd.DataFrame.from_records(test, index=test_id)
df1.to_csv('sub1.csv')

In [ ]:
label_list = []
df1_T = df1.transpose()
for i in range(len(df1_T.columns)):
    label_list.append(df1_T.ix[:, i].idxmax())

In [ ]:
df2 = pd.Series(label_list, index=df1.index)
df2.to_csv('sub2.csv')

In [ ]:
from IPython.display import FileLink
FileLink('sub1.csv')

In [ ]:
FileLink('sub2.csv')